In [1]:
import os
import datetime
import pandas as pd
import numpy as np

# 整理原数据

## 生成时间列表

In [2]:

def create_assist_date(datestart=None, dateend=None):
    # 创建日期辅助表

    if datestart is None:
        datestart = '2011-01-01'
    if dateend is None:
        dateend = datetime.datetime.now().strftime('%Y-%m-%d')
    
    # 转为日期格式
    datestart=datetime.datetime.strptime(datestart,'%Y-%m-%d')
    dateend=datetime.datetime.strptime(dateend,'%Y-%m-%d')
    date_list = []
    date_list.append(datestart.strftime('%Y-%m-%d'))
    while datestart<dateend:
        # 日期叠加一天
        datestart+=datetime.timedelta(days=+1)
        # 日期转字符串存入列表
        date_list.append(datestart.strftime('%Y-%m-%d'))
    return date_list


now_datetime = str(datetime.datetime.now().date())
dateList = create_assist_date(datestart='2011-01-01', dateend=now_datetime)
dataDF = pd.DataFrame(dateList, columns=['时间'])
dataDF.head()

,时间
0,2011-01-01
1,2011-01-02
2,2011-01-03
3,2011-01-04
4,2011-01-05


In [3]:

path = r'F:\爬虫文件夹\百度指数\原始数据'

colu = {'买房_x': '买房', '卖房_x': '卖房', '失业金_x': '失业金', '房价_x': '房价', '招工_x': '招工', '招聘_x': '招聘',
        '租房_x': '租房', '股票_y': '股票'}

city_list = list(set([i.split('_')[0] for i in os.listdir(path)]))
print(city_list[:5], len(city_list))

['巢湖', '抚顺', '昌吉', '晋中', '兴安盟'] 362


In [4]:

for city in city_list:
#     if city != '上海': continue
    df_all = pd.DataFrame()
    for i in range(1, 22):
        path_name = os.path.join(path, '{}_{}.xlsx'.format(city, i))
        print(path_name)
        df = pd.read_excel(path_name)
        df = df.query('type == "all"')
        df['keyword'] = df['keyword'].map(lambda x: x.replace("[{'name': '", '').replace("', 'wordType': 1}]", ""))
        df = df.pivot_table(values='index', columns='keyword', index='date', fill_value=0).reset_index()
        df = pd.merge(left=df, right=dataDF, left_on='date', right_on='时间', how='outer').fillna(0)
        df = df[['时间', '买房', '卖房', '失业金', '房价', '招工', '招聘', '租房', '股票']]
        
        if df_all.shape[0] == 0:
            df_all = df.copy()
        else:
            df_all = df_all.replace(0, np.NaN)
            df_all = pd.merge(df_all, df,on='时间', how='outer')
            df_all = df_all[['时间', '买房_x', '买房_y', '卖房_x', '卖房_y', '失业金_x', '失业金_y', '房价_x', '房价_y','招工_x', '招工_y', '招聘_x', '招聘_y', '租房_x', '租房_y', '股票_x', '股票_y' ]]

            df_all = df_all.fillna(method="bfill",limit=1,axis=1).rename(columns=colu)
            df_all = df_all[['时间', '买房', '卖房', '失业金', '房价','招工', '招聘', '租房', '股票' ]]

    # [['时间', '买房', '卖房', '失业金', '房价','招工', '招聘', '租房', '股票' ]]
    df_all['月'] = df_all['时间'].map(lambda x: x[:7])
    d = []
    for column in [ '买房', '卖房', '失业金', '房价','招工', '招聘', '租房', '股票' ]:
        df_colu = df_all[['月', column]]
        df_colu_gr = df_colu.groupby('月')[[column]].sum()
        d.append(df_colu_gr)
    
    name = '1_百度指数_出布林线_data/{}_百度指数.xlsx'.format(city)
    writer = pd.ExcelWriter(name)
    
    df_all_1 = df_all[['时间', '买房', '卖房', '失业金', '房价','招工', '招聘', '租房', '股票' ]]
    df_all_1.to_excel(writer, sheet_name='原始数据', index=False)

    d1 = pd.concat(d, axis=1).astype(int)    
    for col in d1.columns:
        ddd = pd.DataFrame()
        ddd[col] = d1[col]
        ddd['布林中轨'] = ddd[col].rolling(window=10).mean().fillna(0)
        ddd['布林上轨'] = ddd[col].rolling(window=10).mean().fillna(0) + 2 * ddd[col].rolling(window=10).std().fillna(0)
        ddd['布林下轨'] = ddd[col].rolling(window=10).mean().fillna(0) - 2 * ddd[col].rolling(window=10).std().fillna(0)
        ddd.to_excel(writer, sheet_name=col)
    
    writer.save()
    writer.close()


F:\爬虫文件夹\百度指数\原始数据\巢湖_1.xlsx
F:\爬虫文件夹\百度指数\原始数据\巢湖_2.xlsx
F:\爬虫文件夹\百度指数\原始数据\巢湖_3.xlsx
F:\爬虫文件夹\百度指数\原始数据\巢湖_4.xlsx
F:\爬虫文件夹\百度指数\原始数据\巢湖_5.xlsx
F:\爬虫文件夹\百度指数\原始数据\巢湖_6.xlsx
F:\爬虫文件夹\百度指数\原始数据\巢湖_7.xlsx
F:\爬虫文件夹\百度指数\原始数据\巢湖_8.xlsx
F:\爬虫文件夹\百度指数\原始数据\巢湖_9.xlsx
F:\爬虫文件夹\百度指数\原始数据\巢湖_10.xlsx
F:\爬虫文件夹\百度指数\原始数据\巢湖_11.xlsx
F:\爬虫文件夹\百度指数\原始数据\巢湖_12.xlsx
F:\爬虫文件夹\百度指数\原始数据\巢湖_13.xlsx
F:\爬虫文件夹\百度指数\原始数据\巢湖_14.xlsx
F:\爬虫文件夹\百度指数\原始数据\巢湖_15.xlsx
F:\爬虫文件夹\百度指数\原始数据\巢湖_16.xlsx
F:\爬虫文件夹\百度指数\原始数据\巢湖_17.xlsx
F:\爬虫文件夹\百度指数\原始数据\巢湖_18.xlsx
F:\爬虫文件夹\百度指数\原始数据\巢湖_19.xlsx
F:\爬虫文件夹\百度指数\原始数据\巢湖_20.xlsx
F:\爬虫文件夹\百度指数\原始数据\巢湖_21.xlsx
F:\爬虫文件夹\百度指数\原始数据\抚顺_1.xlsx
F:\爬虫文件夹\百度指数\原始数据\抚顺_2.xlsx
F:\爬虫文件夹\百度指数\原始数据\抚顺_3.xlsx
F:\爬虫文件夹\百度指数\原始数据\抚顺_4.xlsx
F:\爬虫文件夹\百度指数\原始数据\抚顺_5.xlsx
F:\爬虫文件夹\百度指数\原始数据\抚顺_6.xlsx
F:\爬虫文件夹\百度指数\原始数据\抚顺_7.xlsx
F:\爬虫文件夹\百度指数\原始数据\抚顺_8.xlsx
F:\爬虫文件夹\百度指数\原始数据\抚顺_9.xlsx
F:\爬虫文件夹\百度指数\原始数据\抚顺_10.xlsx
F:\爬虫文件夹\百度指数\原始数据\抚顺_11.xlsx
F:\爬虫文件夹\百度指数\原始数据\抚顺_12.xlsx
F:\爬虫文件夹\百度指数\原始数据\抚顺_13.xls

F:\爬虫文件夹\百度指数\原始数据\天水_5.xlsx
F:\爬虫文件夹\百度指数\原始数据\天水_6.xlsx
F:\爬虫文件夹\百度指数\原始数据\天水_7.xlsx
F:\爬虫文件夹\百度指数\原始数据\天水_8.xlsx
F:\爬虫文件夹\百度指数\原始数据\天水_9.xlsx
F:\爬虫文件夹\百度指数\原始数据\天水_10.xlsx
F:\爬虫文件夹\百度指数\原始数据\天水_11.xlsx
F:\爬虫文件夹\百度指数\原始数据\天水_12.xlsx
F:\爬虫文件夹\百度指数\原始数据\天水_13.xlsx
F:\爬虫文件夹\百度指数\原始数据\天水_14.xlsx
F:\爬虫文件夹\百度指数\原始数据\天水_15.xlsx
F:\爬虫文件夹\百度指数\原始数据\天水_16.xlsx
F:\爬虫文件夹\百度指数\原始数据\天水_17.xlsx
F:\爬虫文件夹\百度指数\原始数据\天水_18.xlsx
F:\爬虫文件夹\百度指数\原始数据\天水_19.xlsx
F:\爬虫文件夹\百度指数\原始数据\天水_20.xlsx
F:\爬虫文件夹\百度指数\原始数据\天水_21.xlsx
F:\爬虫文件夹\百度指数\原始数据\上饶_1.xlsx
F:\爬虫文件夹\百度指数\原始数据\上饶_2.xlsx
F:\爬虫文件夹\百度指数\原始数据\上饶_3.xlsx
F:\爬虫文件夹\百度指数\原始数据\上饶_4.xlsx
F:\爬虫文件夹\百度指数\原始数据\上饶_5.xlsx
F:\爬虫文件夹\百度指数\原始数据\上饶_6.xlsx
F:\爬虫文件夹\百度指数\原始数据\上饶_7.xlsx
F:\爬虫文件夹\百度指数\原始数据\上饶_8.xlsx
F:\爬虫文件夹\百度指数\原始数据\上饶_9.xlsx
F:\爬虫文件夹\百度指数\原始数据\上饶_10.xlsx
F:\爬虫文件夹\百度指数\原始数据\上饶_11.xlsx
F:\爬虫文件夹\百度指数\原始数据\上饶_12.xlsx
F:\爬虫文件夹\百度指数\原始数据\上饶_13.xlsx
F:\爬虫文件夹\百度指数\原始数据\上饶_14.xlsx
F:\爬虫文件夹\百度指数\原始数据\上饶_15.xlsx
F:\爬虫文件夹\百度指数\原始数据\上饶_16.xlsx
F:\爬虫文件夹\百度指数\原始数据\上饶_17

F:\爬虫文件夹\百度指数\原始数据\临高_9.xlsx
F:\爬虫文件夹\百度指数\原始数据\临高_10.xlsx
F:\爬虫文件夹\百度指数\原始数据\临高_11.xlsx
F:\爬虫文件夹\百度指数\原始数据\临高_12.xlsx
F:\爬虫文件夹\百度指数\原始数据\临高_13.xlsx
F:\爬虫文件夹\百度指数\原始数据\临高_14.xlsx
F:\爬虫文件夹\百度指数\原始数据\临高_15.xlsx
F:\爬虫文件夹\百度指数\原始数据\临高_16.xlsx
F:\爬虫文件夹\百度指数\原始数据\临高_17.xlsx
F:\爬虫文件夹\百度指数\原始数据\临高_18.xlsx
F:\爬虫文件夹\百度指数\原始数据\临高_19.xlsx
F:\爬虫文件夹\百度指数\原始数据\临高_20.xlsx
F:\爬虫文件夹\百度指数\原始数据\临高_21.xlsx
F:\爬虫文件夹\百度指数\原始数据\安顺_1.xlsx
F:\爬虫文件夹\百度指数\原始数据\安顺_2.xlsx
F:\爬虫文件夹\百度指数\原始数据\安顺_3.xlsx
F:\爬虫文件夹\百度指数\原始数据\安顺_4.xlsx
F:\爬虫文件夹\百度指数\原始数据\安顺_5.xlsx
F:\爬虫文件夹\百度指数\原始数据\安顺_6.xlsx
F:\爬虫文件夹\百度指数\原始数据\安顺_7.xlsx
F:\爬虫文件夹\百度指数\原始数据\安顺_8.xlsx
F:\爬虫文件夹\百度指数\原始数据\安顺_9.xlsx
F:\爬虫文件夹\百度指数\原始数据\安顺_10.xlsx
F:\爬虫文件夹\百度指数\原始数据\安顺_11.xlsx
F:\爬虫文件夹\百度指数\原始数据\安顺_12.xlsx
F:\爬虫文件夹\百度指数\原始数据\安顺_13.xlsx
F:\爬虫文件夹\百度指数\原始数据\安顺_14.xlsx
F:\爬虫文件夹\百度指数\原始数据\安顺_15.xlsx
F:\爬虫文件夹\百度指数\原始数据\安顺_16.xlsx
F:\爬虫文件夹\百度指数\原始数据\安顺_17.xlsx
F:\爬虫文件夹\百度指数\原始数据\安顺_18.xlsx
F:\爬虫文件夹\百度指数\原始数据\安顺_19.xlsx
F:\爬虫文件夹\百度指数\原始数据\安顺_20.xlsx
F:\爬虫文件夹\百度指数\原始数据\安

F:\爬虫文件夹\百度指数\原始数据\嘉兴_12.xlsx
F:\爬虫文件夹\百度指数\原始数据\嘉兴_13.xlsx
F:\爬虫文件夹\百度指数\原始数据\嘉兴_14.xlsx
F:\爬虫文件夹\百度指数\原始数据\嘉兴_15.xlsx
F:\爬虫文件夹\百度指数\原始数据\嘉兴_16.xlsx
F:\爬虫文件夹\百度指数\原始数据\嘉兴_17.xlsx
F:\爬虫文件夹\百度指数\原始数据\嘉兴_18.xlsx
F:\爬虫文件夹\百度指数\原始数据\嘉兴_19.xlsx
F:\爬虫文件夹\百度指数\原始数据\嘉兴_20.xlsx
F:\爬虫文件夹\百度指数\原始数据\嘉兴_21.xlsx
F:\爬虫文件夹\百度指数\原始数据\贵阳_1.xlsx
F:\爬虫文件夹\百度指数\原始数据\贵阳_2.xlsx
F:\爬虫文件夹\百度指数\原始数据\贵阳_3.xlsx
F:\爬虫文件夹\百度指数\原始数据\贵阳_4.xlsx
F:\爬虫文件夹\百度指数\原始数据\贵阳_5.xlsx
F:\爬虫文件夹\百度指数\原始数据\贵阳_6.xlsx
F:\爬虫文件夹\百度指数\原始数据\贵阳_7.xlsx
F:\爬虫文件夹\百度指数\原始数据\贵阳_8.xlsx
F:\爬虫文件夹\百度指数\原始数据\贵阳_9.xlsx
F:\爬虫文件夹\百度指数\原始数据\贵阳_10.xlsx
F:\爬虫文件夹\百度指数\原始数据\贵阳_11.xlsx
F:\爬虫文件夹\百度指数\原始数据\贵阳_12.xlsx
F:\爬虫文件夹\百度指数\原始数据\贵阳_13.xlsx
F:\爬虫文件夹\百度指数\原始数据\贵阳_14.xlsx
F:\爬虫文件夹\百度指数\原始数据\贵阳_15.xlsx
F:\爬虫文件夹\百度指数\原始数据\贵阳_16.xlsx
F:\爬虫文件夹\百度指数\原始数据\贵阳_17.xlsx
F:\爬虫文件夹\百度指数\原始数据\贵阳_18.xlsx
F:\爬虫文件夹\百度指数\原始数据\贵阳_19.xlsx
F:\爬虫文件夹\百度指数\原始数据\贵阳_20.xlsx
F:\爬虫文件夹\百度指数\原始数据\贵阳_21.xlsx
F:\爬虫文件夹\百度指数\原始数据\宣城_1.xlsx
F:\爬虫文件夹\百度指数\原始数据\宣城_2.xlsx
F:\爬虫文件夹\百度指数\原始数据\宣城

F:\爬虫文件夹\百度指数\原始数据\仙桃_16.xlsx
F:\爬虫文件夹\百度指数\原始数据\仙桃_17.xlsx
F:\爬虫文件夹\百度指数\原始数据\仙桃_18.xlsx
F:\爬虫文件夹\百度指数\原始数据\仙桃_19.xlsx
F:\爬虫文件夹\百度指数\原始数据\仙桃_20.xlsx
F:\爬虫文件夹\百度指数\原始数据\仙桃_21.xlsx
F:\爬虫文件夹\百度指数\原始数据\本溪_1.xlsx
F:\爬虫文件夹\百度指数\原始数据\本溪_2.xlsx
F:\爬虫文件夹\百度指数\原始数据\本溪_3.xlsx
F:\爬虫文件夹\百度指数\原始数据\本溪_4.xlsx
F:\爬虫文件夹\百度指数\原始数据\本溪_5.xlsx
F:\爬虫文件夹\百度指数\原始数据\本溪_6.xlsx
F:\爬虫文件夹\百度指数\原始数据\本溪_7.xlsx
F:\爬虫文件夹\百度指数\原始数据\本溪_8.xlsx
F:\爬虫文件夹\百度指数\原始数据\本溪_9.xlsx
F:\爬虫文件夹\百度指数\原始数据\本溪_10.xlsx
F:\爬虫文件夹\百度指数\原始数据\本溪_11.xlsx
F:\爬虫文件夹\百度指数\原始数据\本溪_12.xlsx
F:\爬虫文件夹\百度指数\原始数据\本溪_13.xlsx
F:\爬虫文件夹\百度指数\原始数据\本溪_14.xlsx
F:\爬虫文件夹\百度指数\原始数据\本溪_15.xlsx
F:\爬虫文件夹\百度指数\原始数据\本溪_16.xlsx
F:\爬虫文件夹\百度指数\原始数据\本溪_17.xlsx
F:\爬虫文件夹\百度指数\原始数据\本溪_18.xlsx
F:\爬虫文件夹\百度指数\原始数据\本溪_19.xlsx
F:\爬虫文件夹\百度指数\原始数据\本溪_20.xlsx
F:\爬虫文件夹\百度指数\原始数据\本溪_21.xlsx
F:\爬虫文件夹\百度指数\原始数据\汕头_1.xlsx
F:\爬虫文件夹\百度指数\原始数据\汕头_2.xlsx
F:\爬虫文件夹\百度指数\原始数据\汕头_3.xlsx
F:\爬虫文件夹\百度指数\原始数据\汕头_4.xlsx
F:\爬虫文件夹\百度指数\原始数据\汕头_5.xlsx
F:\爬虫文件夹\百度指数\原始数据\汕头_6.xlsx
F:\爬虫文件夹\百度指数\原始数据\汕头_7.x

...

F:\爬虫文件夹\百度指数\原始数据\喀什_6.xlsx
F:\爬虫文件夹\百度指数\原始数据\喀什_7.xlsx
F:\爬虫文件夹\百度指数\原始数据\喀什_8.xlsx
F:\爬虫文件夹\百度指数\原始数据\喀什_9.xlsx
F:\爬虫文件夹\百度指数\原始数据\喀什_10.xlsx
F:\爬虫文件夹\百度指数\原始数据\喀什_11.xlsx
F:\爬虫文件夹\百度指数\原始数据\喀什_12.xlsx
F:\爬虫文件夹\百度指数\原始数据\喀什_13.xlsx
F:\爬虫文件夹\百度指数\原始数据\喀什_14.xlsx
F:\爬虫文件夹\百度指数\原始数据\喀什_15.xlsx
F:\爬虫文件夹\百度指数\原始数据\喀什_16.xlsx
F:\爬虫文件夹\百度指数\原始数据\喀什_17.xlsx
F:\爬虫文件夹\百度指数\原始数据\喀什_18.xlsx
F:\爬虫文件夹\百度指数\原始数据\喀什_19.xlsx
F:\爬虫文件夹\百度指数\原始数据\喀什_20.xlsx
F:\爬虫文件夹\百度指数\原始数据\喀什_21.xlsx
F:\爬虫文件夹\百度指数\原始数据\黔西南_1.xlsx
F:\爬虫文件夹\百度指数\原始数据\黔西南_2.xlsx
F:\爬虫文件夹\百度指数\原始数据\黔西南_3.xlsx
F:\爬虫文件夹\百度指数\原始数据\黔西南_4.xlsx
F:\爬虫文件夹\百度指数\原始数据\黔西南_5.xlsx
F:\爬虫文件夹\百度指数\原始数据\黔西南_6.xlsx
F:\爬虫文件夹\百度指数\原始数据\黔西南_7.xlsx
F:\爬虫文件夹\百度指数\原始数据\黔西南_8.xlsx
F:\爬虫文件夹\百度指数\原始数据\黔西南_9.xlsx
F:\爬虫文件夹\百度指数\原始数据\黔西南_10.xlsx
F:\爬虫文件夹\百度指数\原始数据\黔西南_11.xlsx
F:\爬虫文件夹\百度指数\原始数据\黔西南_12.xlsx
F:\爬虫文件夹\百度指数\原始数据\黔西南_13.xlsx
F:\爬虫文件夹\百度指数\原始数据\黔西南_14.xlsx
F:\爬虫文件夹\百度指数\原始数据\黔西南_15.xlsx
F:\爬虫文件夹\百度指数\原始数据\黔西南_16.xlsx
F:\爬虫文件夹\百度指数\原始数据\黔西南_17.xlsx
F:\爬虫文

F:\爬虫文件夹\百度指数\原始数据\曲靖_8.xlsx
F:\爬虫文件夹\百度指数\原始数据\曲靖_9.xlsx
F:\爬虫文件夹\百度指数\原始数据\曲靖_10.xlsx
F:\爬虫文件夹\百度指数\原始数据\曲靖_11.xlsx
F:\爬虫文件夹\百度指数\原始数据\曲靖_12.xlsx
F:\爬虫文件夹\百度指数\原始数据\曲靖_13.xlsx
F:\爬虫文件夹\百度指数\原始数据\曲靖_14.xlsx
F:\爬虫文件夹\百度指数\原始数据\曲靖_15.xlsx
F:\爬虫文件夹\百度指数\原始数据\曲靖_16.xlsx
F:\爬虫文件夹\百度指数\原始数据\曲靖_17.xlsx
F:\爬虫文件夹\百度指数\原始数据\曲靖_18.xlsx
F:\爬虫文件夹\百度指数\原始数据\曲靖_19.xlsx
F:\爬虫文件夹\百度指数\原始数据\曲靖_20.xlsx
F:\爬虫文件夹\百度指数\原始数据\曲靖_21.xlsx
F:\爬虫文件夹\百度指数\原始数据\绥化_1.xlsx
F:\爬虫文件夹\百度指数\原始数据\绥化_2.xlsx
F:\爬虫文件夹\百度指数\原始数据\绥化_3.xlsx
F:\爬虫文件夹\百度指数\原始数据\绥化_4.xlsx
F:\爬虫文件夹\百度指数\原始数据\绥化_5.xlsx
F:\爬虫文件夹\百度指数\原始数据\绥化_6.xlsx
F:\爬虫文件夹\百度指数\原始数据\绥化_7.xlsx
F:\爬虫文件夹\百度指数\原始数据\绥化_8.xlsx
F:\爬虫文件夹\百度指数\原始数据\绥化_9.xlsx
F:\爬虫文件夹\百度指数\原始数据\绥化_10.xlsx
F:\爬虫文件夹\百度指数\原始数据\绥化_11.xlsx
F:\爬虫文件夹\百度指数\原始数据\绥化_12.xlsx
F:\爬虫文件夹\百度指数\原始数据\绥化_13.xlsx
F:\爬虫文件夹\百度指数\原始数据\绥化_14.xlsx
F:\爬虫文件夹\百度指数\原始数据\绥化_15.xlsx
F:\爬虫文件夹\百度指数\原始数据\绥化_16.xlsx
F:\爬虫文件夹\百度指数\原始数据\绥化_17.xlsx
F:\爬虫文件夹\百度指数\原始数据\绥化_18.xlsx
F:\爬虫文件夹\百度指数\原始数据\绥化_19.xlsx
F:\爬虫文件夹\百度指数\原始数据\绥化

F:\爬虫文件夹\百度指数\原始数据\三亚_5.xlsx
F:\爬虫文件夹\百度指数\原始数据\三亚_6.xlsx
F:\爬虫文件夹\百度指数\原始数据\三亚_7.xlsx
F:\爬虫文件夹\百度指数\原始数据\三亚_8.xlsx
F:\爬虫文件夹\百度指数\原始数据\三亚_9.xlsx
F:\爬虫文件夹\百度指数\原始数据\三亚_10.xlsx
F:\爬虫文件夹\百度指数\原始数据\三亚_11.xlsx
F:\爬虫文件夹\百度指数\原始数据\三亚_12.xlsx
F:\爬虫文件夹\百度指数\原始数据\三亚_13.xlsx
F:\爬虫文件夹\百度指数\原始数据\三亚_14.xlsx
F:\爬虫文件夹\百度指数\原始数据\三亚_15.xlsx
F:\爬虫文件夹\百度指数\原始数据\三亚_16.xlsx
F:\爬虫文件夹\百度指数\原始数据\三亚_17.xlsx
F:\爬虫文件夹\百度指数\原始数据\三亚_18.xlsx
F:\爬虫文件夹\百度指数\原始数据\三亚_19.xlsx
F:\爬虫文件夹\百度指数\原始数据\三亚_20.xlsx
F:\爬虫文件夹\百度指数\原始数据\三亚_21.xlsx
F:\爬虫文件夹\百度指数\原始数据\文昌_1.xlsx
F:\爬虫文件夹\百度指数\原始数据\文昌_2.xlsx
F:\爬虫文件夹\百度指数\原始数据\文昌_3.xlsx
F:\爬虫文件夹\百度指数\原始数据\文昌_4.xlsx
F:\爬虫文件夹\百度指数\原始数据\文昌_5.xlsx
F:\爬虫文件夹\百度指数\原始数据\文昌_6.xlsx
F:\爬虫文件夹\百度指数\原始数据\文昌_7.xlsx
F:\爬虫文件夹\百度指数\原始数据\文昌_8.xlsx
F:\爬虫文件夹\百度指数\原始数据\文昌_9.xlsx
F:\爬虫文件夹\百度指数\原始数据\文昌_10.xlsx
F:\爬虫文件夹\百度指数\原始数据\文昌_11.xlsx
F:\爬虫文件夹\百度指数\原始数据\文昌_12.xlsx
F:\爬虫文件夹\百度指数\原始数据\文昌_13.xlsx
F:\爬虫文件夹\百度指数\原始数据\文昌_14.xlsx
F:\爬虫文件夹\百度指数\原始数据\文昌_15.xlsx
F:\爬虫文件夹\百度指数\原始数据\文昌_16.xlsx
F:\爬虫文件夹\百度指数\原始数据\文昌_17

F:\爬虫文件夹\百度指数\原始数据\大庆_5.xlsx
F:\爬虫文件夹\百度指数\原始数据\大庆_6.xlsx
F:\爬虫文件夹\百度指数\原始数据\大庆_7.xlsx
F:\爬虫文件夹\百度指数\原始数据\大庆_8.xlsx
F:\爬虫文件夹\百度指数\原始数据\大庆_9.xlsx
F:\爬虫文件夹\百度指数\原始数据\大庆_10.xlsx
F:\爬虫文件夹\百度指数\原始数据\大庆_11.xlsx
F:\爬虫文件夹\百度指数\原始数据\大庆_12.xlsx
F:\爬虫文件夹\百度指数\原始数据\大庆_13.xlsx
F:\爬虫文件夹\百度指数\原始数据\大庆_14.xlsx
F:\爬虫文件夹\百度指数\原始数据\大庆_15.xlsx
F:\爬虫文件夹\百度指数\原始数据\大庆_16.xlsx
F:\爬虫文件夹\百度指数\原始数据\大庆_17.xlsx
F:\爬虫文件夹\百度指数\原始数据\大庆_18.xlsx
F:\爬虫文件夹\百度指数\原始数据\大庆_19.xlsx
F:\爬虫文件夹\百度指数\原始数据\大庆_20.xlsx
F:\爬虫文件夹\百度指数\原始数据\大庆_21.xlsx
F:\爬虫文件夹\百度指数\原始数据\河源_1.xlsx
F:\爬虫文件夹\百度指数\原始数据\河源_2.xlsx
F:\爬虫文件夹\百度指数\原始数据\河源_3.xlsx
F:\爬虫文件夹\百度指数\原始数据\河源_4.xlsx
F:\爬虫文件夹\百度指数\原始数据\河源_5.xlsx
F:\爬虫文件夹\百度指数\原始数据\河源_6.xlsx
F:\爬虫文件夹\百度指数\原始数据\河源_7.xlsx
F:\爬虫文件夹\百度指数\原始数据\河源_8.xlsx
F:\爬虫文件夹\百度指数\原始数据\河源_9.xlsx
F:\爬虫文件夹\百度指数\原始数据\河源_10.xlsx
F:\爬虫文件夹\百度指数\原始数据\河源_11.xlsx
F:\爬虫文件夹\百度指数\原始数据\河源_12.xlsx
F:\爬虫文件夹\百度指数\原始数据\河源_13.xlsx
F:\爬虫文件夹\百度指数\原始数据\河源_14.xlsx
F:\爬虫文件夹\百度指数\原始数据\河源_15.xlsx
F:\爬虫文件夹\百度指数\原始数据\河源_16.xlsx
F:\爬虫文件夹\百度指数\原始数据\河源_17

F:\爬虫文件夹\百度指数\原始数据\东方_10.xlsx
F:\爬虫文件夹\百度指数\原始数据\东方_11.xlsx
F:\爬虫文件夹\百度指数\原始数据\东方_12.xlsx
F:\爬虫文件夹\百度指数\原始数据\东方_13.xlsx
F:\爬虫文件夹\百度指数\原始数据\东方_14.xlsx
F:\爬虫文件夹\百度指数\原始数据\东方_15.xlsx
F:\爬虫文件夹\百度指数\原始数据\东方_16.xlsx
F:\爬虫文件夹\百度指数\原始数据\东方_17.xlsx
F:\爬虫文件夹\百度指数\原始数据\东方_18.xlsx
F:\爬虫文件夹\百度指数\原始数据\东方_19.xlsx
F:\爬虫文件夹\百度指数\原始数据\东方_20.xlsx
F:\爬虫文件夹\百度指数\原始数据\东方_21.xlsx
F:\爬虫文件夹\百度指数\原始数据\荆门_1.xlsx
F:\爬虫文件夹\百度指数\原始数据\荆门_2.xlsx
F:\爬虫文件夹\百度指数\原始数据\荆门_3.xlsx
F:\爬虫文件夹\百度指数\原始数据\荆门_4.xlsx
F:\爬虫文件夹\百度指数\原始数据\荆门_5.xlsx
F:\爬虫文件夹\百度指数\原始数据\荆门_6.xlsx
F:\爬虫文件夹\百度指数\原始数据\荆门_7.xlsx
F:\爬虫文件夹\百度指数\原始数据\荆门_8.xlsx
F:\爬虫文件夹\百度指数\原始数据\荆门_9.xlsx
F:\爬虫文件夹\百度指数\原始数据\荆门_10.xlsx
F:\爬虫文件夹\百度指数\原始数据\荆门_11.xlsx
F:\爬虫文件夹\百度指数\原始数据\荆门_12.xlsx
F:\爬虫文件夹\百度指数\原始数据\荆门_13.xlsx
F:\爬虫文件夹\百度指数\原始数据\荆门_14.xlsx
F:\爬虫文件夹\百度指数\原始数据\荆门_15.xlsx
F:\爬虫文件夹\百度指数\原始数据\荆门_16.xlsx
F:\爬虫文件夹\百度指数\原始数据\荆门_17.xlsx
F:\爬虫文件夹\百度指数\原始数据\荆门_18.xlsx
F:\爬虫文件夹\百度指数\原始数据\荆门_19.xlsx
F:\爬虫文件夹\百度指数\原始数据\荆门_20.xlsx
F:\爬虫文件夹\百度指数\原始数据\荆门_21.xlsx
F:\爬虫文件夹\百度指数\原始数据\